In [1]:
import sys
import os
import csv
from osgeo import ogr
import pandas as pd
from random import uniform

In [2]:
def LoadFile(infile):
    """
    infile is a string with the file name
    Loads a .csv file for a single subject and takes the "practice", lines
    """
    state_db = []
    fin = open(infile, 'U')
    original_db = csv.reader(fin)
    for row in original_db:
        state_db.append(row)
    fin.close()
    return state_db


In [3]:
def Extract_Racial_Data(state_db):
    """
    state_db is a single state database with info about population by race for each cenus block
    this function returns a cropped database with only 5 racial categories: non-hispanic white,
    non-hispanic black, non hispanic asian, hispanic, non hiapanic other, the GISJOIN code, state code, 
    state name, and total population.
    """
    crop_state_db = []
    state_db[0].append('NH_White')
    state_db[0].append('NH_Black')
    state_db[0].append('NH_Asian')
    state_db[0].append('Hispanic')
    state_db[0].append('Other')
    state_db[0].append('Total')
    indGISJOIN = state_db[0].index('GISJOIN')
    indSTATE = state_db[0].index('STATE')
    indSTATEA = state_db[0].index('STATEA')
    indCDA = state_db[0].index('CDA')   #COngressional district
    indPOP = state_db[0].index('H7Z001') #This code is temporary I'm waiting for the real codes from downloaded table
    indPOP_White = state_db[0].index('H7Z003') #These codes are from the original table used to make the new ones
    indPOP_NonLatino = state_db[0].index('H7Z002')
    indPOP_Black = state_db[0].index('H7Z004')
    indPOP_Asian = state_db[0].index('H7Z006')
    indPOP_Hispanic = state_db[0].index('H7Z010')
    indNH_White = state_db[0].index('NH_White')
    indNH_Black = state_db[0].index('NH_Black')
    indNH_Asian = state_db[0].index('NH_Asian')
    indHispanic = state_db[0].index('Hispanic')
    indOther = state_db[0].index('Other')
    indTotal = state_db[0].index('Total')
    crop_state_db = [[state_db[0][indGISJOIN],state_db[0][indSTATE],state_db[0][indSTATEA],
                      state_db[0][indCDA],state_db[0][indNH_White],state_db[0][indNH_Black],state_db[0][indNH_Asian]
                      ,state_db[0][indHispanic],state_db[0][indOther],state_db[0][indTotal]]]
    for line in range(2, len(state_db)):
        tmp = []
        state_db[line].extend([0,0,0,0,0,0])
        state_db[line][indPOP] = int(state_db[line][indPOP])
        state_db[line][indPOP_NonLatino] = int(state_db[line][indPOP_NonLatino])
        state_db[line][indNH_White] = int(state_db[line][indPOP_White])
        state_db[line][indNH_Black] = int(state_db[line][indPOP_Black]) 
        state_db[line][indNH_Asian] = int(state_db[line][indPOP_Asian]) 
        state_db[line][indHispanic] = int(state_db[line][indPOP_Hispanic]) 
        state_db[line][indOther] = int(state_db[line][indPOP_NonLatino]) - int(state_db[line][indPOP_White]) - int(state_db[line][indPOP_Black]) - int(state_db[line][indPOP_Asian])
        state_db[line][indTotal] = int(state_db[line][indPOP])
        tmp = [state_db[line][indGISJOIN],state_db[line][indSTATE],state_db[line][indSTATEA],
                      state_db[line][indCDA],state_db[line][indNH_White],state_db[line][indNH_Black],state_db[line][indNH_Asian]
                      ,state_db[line][indHispanic],state_db[line][indOther],state_db[line][indTotal]]
        crop_state_db.append(tmp)
    return crop_state_db

In [4]:
def Make_CSV_Out(crop_state_db):
    """
    Take crop_state_db as input and create a csv file with the relevant data
    """
    IndState_Code = crop_state_db[0].index('STATEA')
    state_code = crop_state_db[1][IndState_Code]
    name = '%s/cropped_state_%s_test.csv' % (filepath, state_code)
    fou = open(name, 'w')
    datawriter = csv.writer(fou, dialect='excel')
    for row in crop_state_db:
        datawriter.writerow(row)
    fou.close()

In [5]:
filepath = '/Users/frankanayet/Documents/Research/Personal/Maps'
#os.chdir(filepath)
sys.path.append(filepath)
#state = ['1', '15', '32']
#state = ['12', '17', '36', '39', '42', '48', '4', '5', '8',
#          '9', '13', '16', '18', '19', '20', '21', '22', '23', '24',
#         '25', '26', '27', '28', '29', '31', '33', '34', '35', '37', '40']
#state = ['41', '45', '47', '49', '51', '53', '54', '55']
state = ['44']
state_dict = {'1':'AL','2':'AK','4':'AZ','5':'AR','6':'CA','8':'CO','9':'CT','10':'DE',
              '11':'DC','12':'FL','13':'GA','15':'HI','16':'ID','17':'IL','18':'IN',
              '19':'IA','20':'KS','21':'KY','22':'LA','23':'ME','24':'MD','25':'MA',
              '26':'MI','27':'MN','28':'MS','29':'MO','30':'MT','31':'NE','32':'NV',
              '33':'NH','34':'NJ','35':'NM','36':'NY','37':'NC','38':'ND','39':'OH',
              '40':'OK','41':'OR','42':'PA','44':'RI','45':'SC','46':'SD','47':'TN',
              '48':'TX','49':'UT','50':'VT','51':'VA','53':'WA','54':'WV','55':'WI',
              '56':'WY'}

In [6]:
pwd

u'/Users/frankanayet/Documents/Research/Personal/Maps'

In [7]:
st = '44'

In [8]:
infile = 'nhgis0005_csv/nhgis0005_ds172_2010_block_'+st+'.csv'
state_db = LoadFile(infile)
crop_state_db = Extract_Racial_Data(state_db)
Make_CSV_Out(crop_state_db)
IndState_Code = crop_state_db[0].index('STATEA')
state_code = crop_state_db[1][IndState_Code] # this will be generalized when there is a list of states

In [9]:
infile

'nhgis0005_csv/nhgis0005_ds172_2010_block_44.csv'

In [10]:
len(crop_state_db)

25182

In [11]:
IndState_Code = crop_state_db[0].index('STATEA')

In [12]:
IndState_Code

2

In [13]:
state_code = crop_state_db[1][IndState_Code]

In [14]:
state_code

'44'

In [15]:
crop_file = 'cropped_state_%s.csv' % (state_code)

In [16]:
fin = open(crop_file, 'U')
line_reader = csv.reader(fin)
counter = 0
GIStoIndex = {}
IndexToGIS = {}

In [17]:
for row in line_reader:
    if not (row[0] in GIStoIndex):
        GIStoIndex[row[0]] = counter
        IndexToGIS[counter] = row[0]
    counter += 1
fin.close() 

In [18]:
len(GIStoIndex)

25182

In [19]:
len(IndexToGIS)

25182

In [20]:
state_name = state_dict[st]
ds = ogr.Open('nhgis0005_shapefile_tl2010_'+st+'0_block_2010/'+state_name+'_block_2010.shp')
if ds is None:
    print "Open failed.\n"
    sys.exit(1)

In [21]:
lyr = ds.GetLayerByIndex(0)

In [22]:
len(lyr)

24738

In [23]:
lyr.ResetReading()
    
feat_defn = lyr.GetLayerDefn()
    
field_defns = [feat_defn.GetFieldDefn(i) for i in range(feat_defn.GetFieldCount())]

In [24]:
n_features = len(lyr)
n_features

24738

In [25]:
lyr[0].GetField('GISJOIN') #same code as crop file
lyr[0].GetField('STATEFP10') #state code
lyr[0].GetField('ALAND10')  #LAND AREA?
lyr[0].GetField('AWATER10') #WATER AREA?
lyr[0].GetField('INTPTLAT10') #LATITUDE?
lyr[0].GetField('INTPTLON10') #LONGITUDE?
lyr[0].GetField('Shape_area')
lyr[0].GetField('Shape_len')

650.310510365

In [26]:
 lyr[0].GetField('GISJOIN')

'G44000100304002022'

In [27]:
pop_dat = pd.read_csv(crop_file)
#xlon,xlat and mx,my are not necessarily the same point but are central points inside the block
x_y_dataFields = ['GISJOIN', 'State', 'ConDis', 'mx', 'my']
x_y_data = []
x_y_data.append(x_y_dataFields)

In [28]:
x_y_data

[['GISJOIN', 'State', 'ConDis', 'mx', 'my']]

In [29]:
GIStoIndex['G44000300207011009']

2104

In [30]:
import glob as glob

In [31]:
states_xy = glob.glob('/Users/frankanayet/Desktop/Maps/xx_y*.csv')
states_xy

['/Users/frankanayet/Desktop/Maps/xx_y_state_44_NAD83_FULL.csv']

In [32]:
states_shp = glob.glob('nhgis0005_shapefile_tl2010_*_block_2010/*.shp')
states_shp

['nhgis0005_shapefile_tl2010_10_block_2010/AL_block_2010.shp',
 'nhgis0005_shapefile_tl2010_120_block_2010/FL_block_2010.shp',
 'nhgis0005_shapefile_tl2010_130_block_2010/GA_block_2010.shp',
 'nhgis0005_shapefile_tl2010_150_block_2010/HI_block_2010.shp',
 'nhgis0005_shapefile_tl2010_160_block_2010/ID_block_2010.shp',
 'nhgis0005_shapefile_tl2010_170_block_2010/IL_block_2010.shp',
 'nhgis0005_shapefile_tl2010_180_block_2010/IN_block_2010.shp',
 'nhgis0005_shapefile_tl2010_190_block_2010/IA_block_2010.shp',
 'nhgis0005_shapefile_tl2010_200_block_2010/KS_block_2010.shp',
 'nhgis0005_shapefile_tl2010_210_block_2010/KY_block_2010.shp',
 'nhgis0005_shapefile_tl2010_220_block_2010/LA_block_2010.shp',
 'nhgis0005_shapefile_tl2010_230_block_2010/ME_block_2010.shp',
 'nhgis0005_shapefile_tl2010_240_block_2010/MD_block_2010.shp',
 'nhgis0005_shapefile_tl2010_250_block_2010/MA_block_2010.shp',
 'nhgis0005_shapefile_tl2010_260_block_2010/MI_block_2010.shp',
 'nhgis0005_shapefile_tl2010_270_block_20

In [33]:
states = ['1','4','5','6','8','9',
              '12','13','15','16','17','18',
              '19','20','21','22','23','24','25',
              '26','27','28','29','31','32',
              '33','34','35','36','37','39',
              '40','41','42','44','45','47',
              '48','49','51','53','54','55']

In [34]:
db = pd.read_csv(states_xy[7])

IndexError: list index out of range

In [35]:
len(db.GISJOIN.unique())

NameError: name 'db' is not defined

In [46]:
GIS_diff = []
crop_diff = []
for st in states:
    print st
    state_shp = 'nhgis0005_shapefile_tl2010_'+st+'0_block_2010/'+state_dict[st]+'_block_2010.shp'
    state_xy = 'xx_y_state_'+st+'_NAD83_FULL.csv'
    if len(st) == 1:
        state_crop = 'cropped_state_0'+st+'.csv'
    else:
        state_crop = 'cropped_state_'+st+'.csv'
    ds = ogr.Open(state_shp)
    if ds is None:
        print "Open failed.\n"
        sys.exit(1)
    db = pd.read_csv(state_xy)
    db_crop = pd.read_csv(state_crop)
    xy_len = len(db.GISJOIN.unique())
    crop_len = len(db_crop.GISJOIN.unique())
    lyr = ds.GetLayerByIndex(0)
    shp_len = len(lyr)
    print shp_len
    xy_shp = xy_len - shp_len
    GIS_diff.append(xy_shp)
    crop_diff.append(crop_len-shp_len)
    del db
    del ds
    del db_crop
    

1
251720
4
241666
5
186211
6
708078
8
201062
9
66966
12
477683
13
290406
15
23979
16
149842
17
451426
18
267014
19
216007
20
238600
21
161672
22
203447
23
67681
24
140563
25
155463
26
327855
27
259630
28
171339
29
343565
31
193352
32
84538
33
48758
34
168199
35
168609
36
347985
37
286395
39
365056
40
269118
41
195918
42
421471
44
24738
45
180221
47
240116
48
912889
49
115406
51
281230
53
192211
54
135218
55
252505


In [47]:
GIS_diff

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0]

In [48]:
crop_diff

[546,
 0,
 0,
 2067,
 0,
 612,
 6798,
 680,
 1037,
 0,
 128,
 57,
 0,
 0,
 0,
 1000,
 1837,
 4684,
 2045,
 2030,
 147,
 439,
 0,
 0,
 0,
 79,
 1389,
 0,
 2184,
 2592,
 288,
 0,
 703,
 74,
 443,
 1687,
 0,
 1342,
 0,
 4532,
 3363,
 0,
 591]

In [49]:
state_crop

'cropped_state_55.csv'

In [36]:
db_crop = pd.read_csv('cropped_state_01.csv')

In [37]:
db_crop.head()

,GISJOIN,STATE,STATEA,CDA,NH_White,NH_Black,NH_Asian,Hispanic,Other,Total
0,G01000100201001000,Alabama,1,2,55,4,1,0,1,61
1,G01000100201001001,Alabama,1,2,0,0,0,0,0,0
2,G01000100201001002,Alabama,1,2,0,0,0,0,0,0
3,G01000100201001003,Alabama,1,2,66,4,0,4,1,75
4,G01000100201001004,Alabama,1,2,0,0,0,0,0,0


In [38]:
db_crop.loc[db_crop['Total']==0].count()

GISJOIN     116827
STATE       116827
STATEA      116827
CDA         116827
NH_White    116827
NH_Black    116827
NH_Asian    116827
Hispanic    116827
Other       116827
Total       116827
dtype: int64

In [39]:
db = pd.read_csv('/Users/frankanayet/Desktop/Maps/x_y_state_1_NAD83_FULL.csv')

In [40]:
len(db.GISJOIN.unique())

135439

In [41]:
len(db_crop.GISJOIN.unique())

252266

In [42]:
len(db_crop.GISJOIN.unique()) - len(db.GISJOIN.unique())

116827

In [43]:
del db
del db_crop

1. Merge clustered and x_y
2. Make database with only unique GIS codes (no repeats)
3. Merge x_y and xx_y (adding only new GIS codes)
4. Assign Labels to new GIS codes

In [44]:
clus = pd.read_csv('Clustered_1.csv')

In [45]:
xy = pd.read_csv('x_y_state_1_NAD83_FULL.csv')

In [46]:
clus.head()

,Unnamed: 0,mx,my,ConDis,Labels
0,0,823716.015433,-835647.717312,1,3
1,1,822875.722766,-835862.318793,1,3
2,2,822885.535964,-835477.182424,1,3
3,3,823554.648886,-836866.628923,1,3
4,4,823323.757644,-834970.681825,1,3


In [47]:
xy.head()

,GISJOIN,State,ConDis,mx,my
0,G01000100211002002,1,2,823716.015433,-835647.717312
1,G01000100211002002,1,2,822875.722766,-835862.318793
2,G01000100211002002,1,2,822885.535964,-835477.182424
3,G01000100211002002,1,2,823554.648886,-836866.628923
4,G01000100211002002,1,2,823323.757644,-834970.681825


In [48]:
test = pd.concat([xy,clus], axis=1)

In [49]:
test.head()

,GISJOIN,State,ConDis,mx,my,Unnamed: 0,mx,my,ConDis,Labels
0,G01000100211002002,1,2,823716.015433,-835647.717312,0,823716.015433,-835647.717312,1,3
1,G01000100211002002,1,2,822875.722766,-835862.318793,1,822875.722766,-835862.318793,1,3
2,G01000100211002002,1,2,822885.535964,-835477.182424,2,822885.535964,-835477.182424,1,3
3,G01000100211002002,1,2,823554.648886,-836866.628923,3,823554.648886,-836866.628923,1,3
4,G01000100211002002,1,2,823323.757644,-834970.681825,4,823323.757644,-834970.681825,1,3


In [50]:
test.keys()

Index([u'GISJOIN', u'State', u'ConDis', u'mx', u'my', u'Unnamed: 0', u'mx',
       u'my', u'ConDis', u'Labels'],
      dtype='object')

# STEP 1

In [51]:
test_M = pd.merge(xy,clus[['ConDis','Labels']],how='outer',left_index=True,right_index=True,suffixes=('_xy','_clus'))

In [52]:
test_M.head()

,GISJOIN,State,ConDis_xy,mx,my,ConDis_clus,Labels
0,G01000100211002002,1,2,823716.015433,-835647.717312,1,3
1,G01000100211002002,1,2,822875.722766,-835862.318793,1,3
2,G01000100211002002,1,2,822885.535964,-835477.182424,1,3
3,G01000100211002002,1,2,823554.648886,-836866.628923,1,3
4,G01000100211002002,1,2,823323.757644,-834970.681825,1,3


In [53]:
(test_M.ConDis_xy - test_M.ConDis_clus).unique()

array([1])

# STEP 2 & 3

In [54]:
xxy = pd.read_csv('xx_y_state_1_NAD83_FULL.csv')

Can i group first???

In [55]:
test_M = test_M.rename(columns={'ConDis_xy':'ConDis'})

In [56]:
test_M.head()

,GISJOIN,State,ConDis,mx,my,ConDis_clus,Labels
0,G01000100211002002,1,2,823716.015433,-835647.717312,1,3
1,G01000100211002002,1,2,822875.722766,-835862.318793,1,3
2,G01000100211002002,1,2,822885.535964,-835477.182424,1,3
3,G01000100211002002,1,2,823554.648886,-836866.628923,1,3
4,G01000100211002002,1,2,823323.757644,-834970.681825,1,3


Instead of doinf this long merge that takes a long time divide in two steps to groups one with mean for mx and my, one for the rest with mode and finally merge two tables together.

In [63]:
group_mx = test_M[['GISJOIN','mx','my']].groupby('GISJOIN').mean()

In [58]:
group_labels = test_M[['GISJOIN','State','ConDis','ConDis_clus'
                       ,'Labels']].groupby('GISJOIN').agg(lambda x:x.value_counts().index[0])

In [67]:
test_grouped2 = pd.concat([group_mx, group_labels], axis=1)

In [68]:
test_grouped2.head()

,mx,my,State,ConDis,ConDis_clus,Labels
GISJOIN,,,,,,
G01000100201001000,852302.864369,-839886.634163,1,2,1,3
G01000100201001003,851692.013686,-840059.221633,1,2,1,3
G01000100201001005,851722.960376,-839950.956276,1,2,1,3
G01000100201001007,850601.906345,-839909.356047,1,2,1,3
G01000100201001009,850112.761817,-839274.668730,1,2,1,3


In [72]:
test_grouped2.Labels.unique()

array([3, 6, 0, 1, 5, 2, 4])

In [98]:
#test_grouped = test_M.groupby('GISJOIN').agg(lambda x:x.value_counts().index[0])

In [99]:
#test_grouped.head()

,State,ConDis,mx,my,ConDis_clus,Labels
GISJOIN,,,,,,
G01000100201001000,1,2,852610.187133,-839657.563373,1,3
G01000100201001003,1,2,851949.874980,-840250.851796,1,3
G01000100201001005,1,2,851722.960376,-839950.956276,1,3
G01000100201001007,1,2,850706.293495,-840224.758491,1,3
G01000100201001009,1,2,850112.761817,-839274.668730,1,3


In [100]:
#test_grouped.Labels.unique()

array([3, 6, 0, 1, 5, 2, 4])

In [153]:
#test_grouped.Labels = test_grouped.Labels.round()

In [101]:
#test_grouped.Labels.unique()

array([3, 6, 0, 1, 5, 2, 4])

In [73]:
len(test_grouped2)

135439

In [76]:
del group_labels
del group_mx

In [77]:
xxy.head()

,GISJOIN,State,ConDis,mx,my
0,G01000100211002002,1,2,824217.214937,-835613.626000
1,G01000100211002002,1,2,822261.722693,-836238.815420
2,G01000100211002002,1,2,824791.260661,-833888.771854
3,G01000100211002002,1,2,823367.673138,-835382.793859
4,G01000100211002002,1,2,822031.870421,-836671.942741


In [78]:
xxy_grouped = xxy.groupby('GISJOIN').mean()

In [79]:
len(xxy_grouped)

251720

In [80]:
xxy_grouped.head()

,State,ConDis,mx,my
GISJOIN,,,,
G01000100201001000,1,2,852355.434355,-839799.221066
G01000100201001001,1,2,852625.565825,-840169.063698
G01000100201001002,1,2,851772.531718,-839803.107972
G01000100201001003,1,2,851634.733810,-840017.212934
G01000100201001004,1,2,852574.321524,-840589.038780


In [81]:
xxy_grouped.ConDis.unique()

array([2, 1, 6, 4, 3, 7, 5])

In [82]:
len(xxy) - len(xxy_grouped)

4644297

In [84]:
len(xxy_grouped) - len(test_grouped2)

116281

In [86]:
test_new = pd.concat([test_grouped2,xxy_grouped])

In [87]:
len(test_new)

387159

In [88]:
test_new.head()

,ConDis,ConDis_clus,Labels,State,mx,my
GISJOIN,,,,,,
G01000100201001000,2,1,3,1,852302.864369,-839886.634163
G01000100201001003,2,1,3,1,851692.013686,-840059.221633
G01000100201001005,2,1,3,1,851722.960376,-839950.956276
G01000100201001007,2,1,3,1,850601.906345,-839909.356047
G01000100201001009,2,1,3,1,850112.761817,-839274.668730


In [89]:
len(test_new.groupby(level=0).mean())

251720

In [90]:
test_new.ConDis.unique()

array([2, 1, 6, 4, 3, 7, 5])

In [92]:
test_new.Labels.unique()

array([  3.,   6.,   0.,   1.,   5.,   2.,   4.,  nan])

In [91]:
test_new.ConDis_clus.unique()

array([  1.,   0.,   5.,   3.,   2.,   6.,   4.,  nan])

In [93]:
import numpy as np

In [98]:
test_new.loc[np.isnan(test_new['Labels']), 'ConDis_clus'].unique()

array([ nan])

In [99]:
test_new.loc[np.isnan(test_new['ConDis_clus']), 'Labels'].unique()

array([ nan])

In [105]:
np.isnan(test_new.ConDis).unique()

array([False], dtype=object)

In [117]:
test_new.loc[np.isnan(test_new['ConDis_clus']), 'Labels']

GISJOIN
G01000100201001000   NaN
G01000100201001001   NaN
G01000100201001002   NaN
G01000100201001003   NaN
G01000100201001004   NaN
G01000100201001005   NaN
G01000100201001006   NaN
G01000100201001007   NaN
G01000100201001008   NaN
G01000100201001009   NaN
G01000100201001010   NaN
G01000100201001011   NaN
G01000100201001012   NaN
G01000100201001013   NaN
G01000100201001014   NaN
G01000100201001015   NaN
G01000100201001016   NaN
G01000100201001017   NaN
G01000100201001018   NaN
G01000100201001019   NaN
G01000100201001020   NaN
G01000100201001021   NaN
G01000100201001022   NaN
G01000100201001023   NaN
G01000100201001024   NaN
G01000100201001025   NaN
G01000100201001026   NaN
G01000100201001027   NaN
G01000100201002000   NaN
G01000100201002001   NaN
                      ..
G01013309659003076   NaN
G01013309659003077   NaN
G01013309659003078   NaN
G01013309659003079   NaN
G01013309659003080   NaN
G01013309659003081   NaN
G01013309659003082   NaN
G01013309659003083   NaN
G0101330965900308

# Step 4

In [108]:
centroids = test_new[['Labels','mx','my']].groupby('Labels').mean()

In [109]:
centroids

,mx,my
Labels,,
0,920497.494211,-909528.325183
1,809319.630715,-720488.237531
2,842581.785350,-595203.362274
3,775751.040133,-813164.945150
4,745734.048895,-621112.262495
5,893265.133802,-711245.963453
6,742698.199456,-1040632.901990


In [112]:
to_label = test_new.loc[np.isnan(test_new['Labels']), ['mx','my','Labels']]

In [113]:
len(to_label)

251720

In [115]:
to_label.Labels.unique()

array([ nan])

In [119]:
from scipy.spatial import distance as dist

In [120]:
res = dist.cdist(centroids,to_label[['mx','my']])

In [121]:
res.shape

(7, 251720)

In [122]:
to_label['Labels'] = res.argmin(0)

In [123]:
to_label.head()

,mx,my,Labels
GISJOIN,,,
G01000100201001000,852355.434355,-839799.221066,3
G01000100201001001,852625.565825,-840169.063698,3
G01000100201001002,851772.531718,-839803.107972,3
G01000100201001003,851634.733810,-840017.212934,3
G01000100201001004,852574.321524,-840589.038780,3


In [124]:
to_label.Labels.unique()

array([3, 0, 6, 1, 5, 2, 4])

In [125]:
final_db = test_new.copy()

In [129]:
final_db.head(10)

,ConDis,ConDis_clus,Labels,State,mx,my
GISJOIN,,,,,,
G01000100201001000,2,1,3,1,852302.864369,-839886.634163
G01000100201001003,2,1,3,1,851692.013686,-840059.221633
G01000100201001005,2,1,3,1,851722.960376,-839950.956276
G01000100201001007,2,1,3,1,850601.906345,-839909.356047
G01000100201001009,2,1,3,1,850112.761817,-839274.668730
G01000100201001012,2,1,3,1,850839.478406,-840425.338462
G01000100201001013,2,1,3,1,850735.917925,-840424.238166
G01000100201001014,2,1,3,1,852486.911475,-840324.717015
G01000100201001015,2,1,3,1,852752.705652,-840354.746806


In [132]:
len(final_db.loc[np.isnan(final_db['Labels']),'Labels'])

251720

In [133]:
len(to_label.Labels)

251720

In [134]:
len(final_db)

387159

In [135]:
final_db.Labels.unique()

array([  3.,   6.,   0.,   1.,   5.,   2.,   4.,  nan])

In [136]:
final_db.loc[np.isnan(final_db['Labels']),'Labels'] = to_label.Labels

In [137]:
final_db.head(30)

,ConDis,ConDis_clus,Labels,State,mx,my
GISJOIN,,,,,,
G01000100201001000,2,1,3,1,852302.864369,-839886.634163
G01000100201001003,2,1,3,1,851692.013686,-840059.221633
G01000100201001005,2,1,3,1,851722.960376,-839950.956276
G01000100201001007,2,1,3,1,850601.906345,-839909.356047
G01000100201001009,2,1,3,1,850112.761817,-839274.668730
G01000100201001012,2,1,3,1,850839.478406,-840425.338462
G01000100201001013,2,1,3,1,850735.917925,-840424.238166
G01000100201001014,2,1,3,1,852486.911475,-840324.717015
G01000100201001015,2,1,3,1,852752.705652,-840354.746806


In [138]:
len(final_db)

387159

In [139]:
final_db.Labels.unique()

array([ 3.,  6.,  0.,  1.,  5.,  2.,  4.])

In [140]:
final_db.ConDis = final_db.ConDis - 1

In [141]:
final_db.head()

,ConDis,ConDis_clus,Labels,State,mx,my
GISJOIN,,,,,,
G01000100201001000,1,1,3,1,852302.864369,-839886.634163
G01000100201001003,1,1,3,1,851692.013686,-840059.221633
G01000100201001005,1,1,3,1,851722.960376,-839950.956276
G01000100201001007,1,1,3,1,850601.906345,-839909.356047
G01000100201001009,1,1,3,1,850112.761817,-839274.668730


In [149]:
len(final_db.loc[np.isnan(final_db['ConDis_clus']),'Labels'])

251720

In [150]:
final_db.sort_values('Labels',inplace=True)

In [151]:
final_db.head()

,ConDis,ConDis_clus,Labels,State,mx,my
GISJOIN,,,,,,
G01004500204001016,1,NaN,0,1,938871.673690,-945658.375042
G01010100015003018,2,NaN,0,1,870052.093899,-850085.883873
G01010100015003019,2,NaN,0,1,870002.948517,-850159.232783
G01010100015003020,2,NaN,0,1,869766.652842,-850373.132174
G01010100015003021,2,NaN,0,1,870197.458955,-850355.958182
